Domino's Chat to Cart MVE

In [274]:
import os
import requests
import json
import openai
import random
import datetime

openai.api_key = 'bca45c7dde194c1c8bbe124e8e991d5b'
openai.api_base = 'https://dpz-openai.openai.azure.com/'
openai.api_type = 'azure'
openai.api_version = '2023-05-15'
deployment_name='dpz-sandbox'
model_version = 'gpt-35-turbo'
temperature_level = 0.5
total_tokens = 0

In [275]:
try:
    with open('order_schema.json', 'r') as file:
        order_schema = file.read()
except FileNotFoundError:
    print("The file was not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")


In [276]:
try:
    with open('menu.txt', 'r') as file:
        menu = file.read()
except FileNotFoundError:
    print("The file was not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")


In [277]:
order_complexity = [
    "very simple",
    "simple",
    "average",
    "somewhat complex",
    "very complex"
]

complexity = random.choice(order_complexity)

guests_served = [
    "one person for lunch",
    "a family of three for dinner",
    "a family of six for dinner",
    "an office party for ten people",
    "a soccer tournament for twenty teenagers"
]

guests = random.choice(guests_served)

ground_truth_json_assistant_system_message = f"""You are a test data generator for domino's test orders. 
                Your response should only include a valid json object with no additional text or comments.
                Using the menu in triple backticks ```{menu}``` create an order.
                The order should be appropriate to feed {guests}. The amount of food order should be based on the guests being served.
                The complexity of the order includes things like the number and variety of items ordered as well as special requests.
                The complexity for this sample order should be {complexity}. The complexity should not affect the amount of food order, just the complexity of the choices and options.
                The output should be exactly in the format provided by the json schema in triple backticks ```{order_schema}```
                """

ground_truth_test_order = openai.ChatCompletion.create(
    engine = deployment_name,
    model = model_version,
    messages = [{"role": "assistant", "content": ground_truth_json_assistant_system_message}],
    temperature = temperature_level
)
total_tokens = total_tokens + ground_truth_test_order.usage.total_tokens

print('Test order to serve ' + guests + ' that is ' + complexity + ':\n\n' + ground_truth_test_order.choices[0].message.content)

Test order to serve one person for lunch that is somewhat complex:

{
    "order": {
        "items": [
            {
                "itemId": 1,
                "itemType": "pizza",
                "itemName": "ExtravaganZZa",
                "quantity": 1,
                "options": {
                    "crustType": "Hand Tossed",
                    "sauceType": "Robust Inspired Tomato Sauce",
                    "toppings": "Pepperoni, Ham, Italian Sausage, Beef, Fresh Onions, Fresh Green Peppers, Fresh Mushrooms, Black Olives, Provolone Cheese, Mozzarella Cheese"
                },
                "size": "Medium",
                "specialRequest": "Cut in squares"
            },
            {
                "itemId": 2,
                "itemType": "loaded tots",
                "itemName": "Philly Cheese Steak Loaded Tots",
                "quantity": 1,
                "options": {},
                "size": "Regular",
                "specialRequest": "Extra crispy"
         

In [278]:
order_detail = [
    "vague and missing some details",
    "normal, but don't include common things like the type of sauce on a pizza",
    "very specific include all details"
]

detail = random.choice(order_complexity)

max_turns_per_order_before_fail = 15
starting_message = "Welcome to Domino's, how can I help you?"
customer_system_message = f"""You are a human ordering food from Domino's 
                You are trying to place the specific order that is in the json object in triple backticks ```{ground_truth_test_order.choices[0].message.content}```
                In your initial message to the Domino's online ordering chatbot your message should be {detail}
                Ask for your order using sentences or partial sentences in a conversatioal tone.
                Do not use bulleted lists with each item on a seprate line.
                You should answer any questions to clarify your order using the information from the json object sample order above.
                """
customer_message_memory = [{"role": "system", "content": customer_system_message}, {"role": "user", "content": starting_message}]

dominos_system_message = f"""You are a domino's pizza order chatbot. Engage with the customer until you are confident you have their order correct.
                Then output their order in a JSON format so it can be added to a cart via Domino's API using the following JSON schema {order_schema}
                """
dominos_message_memory = [{"role": "system", "content": dominos_system_message}]

print("\nDomino's: " + starting_message)

order_succeeded = False

for i in range(max_turns_per_order_before_fail):

    response_customer = openai.ChatCompletion.create(
        engine = deployment_name,
        model = model_version,
        messages = customer_message_memory,
        temperature = temperature_level
    )
    total_tokens = total_tokens + response_customer.usage.total_tokens

    print('\nCustomer: ' + response_customer.choices[0].message.content)

    customer_message_memory.append({"role": "assistant", "content": response_customer.choices[0].message.content})
    dominos_message_memory.append({"role": "user", "content": response_customer.choices[0].message.content})

    response_order_bot = openai.ChatCompletion.create(
        engine = deployment_name,
        model = model_version,
        messages = dominos_message_memory,
        temperature = temperature_level
    )
    total_tokens = total_tokens + response_order_bot.usage.total_tokens

    print('\nDomino\'s: ' + response_order_bot.choices[0].message.content)

    dominos_message_memory.append({"role": "assistant", "content": response_order_bot.choices[0].message.content})
    customer_message_memory.append({"role": "user", "content": response_order_bot.choices[0].message.content})

    if (response_order_bot.choices[0].message.content.find('"order"') > -1):

        evaluation_assistant_system_message = f"""You are a data quality assurance assistant. 
                        Compare two messages with JSON objects.
                        The ground truth object is {ground_truth_test_order.choices[0].message.content}
                        The test object is {response_order_bot.choices[0].message.content}
                        Carefully review the data in each object.
                        In detail, provide any diffeences between the two.
                        Indicate if the difference is significant or insignificant to the overall meaning of the two objects.
                        For example, if the size of a pizza is incorrect that would be significant.
                        If a proprty contains an empty string and the test proprty contains words line none or null, that is not significant.
                        Use your best judgement and expert skills to provide a very accurate response.
                        """

        evaluation_result = openai.ChatCompletion.create(
            engine = deployment_name,
            model = model_version,
            messages = [{"role": "assistant", "content": evaluation_assistant_system_message}],
            temperature = temperature_level
        )
        total_tokens = total_tokens + evaluation_result.usage.total_tokens

        print("\n\nEvaluation:\n\n" + evaluation_result.choices[0].message.content)
        break



Domino's: Welcome to Domino's, how can I help you?

Customer: Hi, I would like to place an order please.

Domino's: Sure thing! What would you like to order today?

Customer: I would like to order an ExtravaganZZa pizza, Philly Cheese Steak Loaded Tots, Crispy Bacon & Tomato Specialty Chicken, Chicken Caesar Salad, Coke, and Domino's Marbled Cookie Brownie please.

Domino's: Great choices! What size would you like your ExtravaganZZa pizza to be? We have small, medium, large, and extra large.

Customer: I would like the ExtravaganZZa pizza to be a medium size please.

Domino's: Sounds good! What crust type would you like for your pizza? We have Hand Tossed, Thin Crust, Brooklyn Style, Gluten Free, and Handmade Pan.

Customer: I would like the Hand Tossed crust type for my pizza please.

Domino's: Got it! And what sauce type would you like for your pizza? We have Robust Inspired Tomato Sauce, Hearty Marinara Sauce, BBQ Sauce, Garlic Parmesan Sauce, and Alfredo Sauce.

Customer: I would 

Timeout: Request timed out: HTTPSConnectionPool(host='dpz-openai.openai.azure.com', port=443): Read timed out. (read timeout=600)

In [ ]:
filename = 'evaluations.txt'

current_timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

eval_stirng = f'{current_timestamp} evaluation:\n {evaluation_result.choices[0].message.content}\n\n'

# Open the file in append mode and write the string
with open(filename, 'a') as file:
    file.write(eval_stirng)


In [ ]:
try:
    with open('evaluations.txt', 'r') as file:
        evaluations = file.read()
except FileNotFoundError:
    print("The file was not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")

summary_assistant_system_message = f"""You are an expert QA executive assistant. 
                Analyze the evaluations in this file {evaluations}
                Each evaluation is marked with a timestamp when it was conducted.
                Carefully review each test and find common patterns among the tests.
                Write a professional report in a valid markdown file format that can be shared with executive stakeholders.
                Include the following:
                1) the number of evaluations conducted
                2) in a table format that includes a row with totals at the bottom, the % that contained only insignificant differences vs. those that contain significant differences that would impact the order
                3) provide an expert analysis of the five most common significant differences that wold impact an order and potentially upset a customer
                """

summary_result = openai.ChatCompletion.create(
    engine = deployment_name,
    model = model_version,
    messages = [{"role": "assistant", "content": summary_assistant_system_message}],
    temperature = temperature_level
)
total_tokens = total_tokens + summary_result.usage.total_tokens

# Open the file in append mode and write the string
with open("summary.md", 'w') as file:
    file.write(summary_result.choices[0].message.content)


In [ ]:
print(f'Total tokens: {total_tokens}')

Total tokens: 19019
